In [10]:
import pandas as pd
from py2neo import Graph
import os
import json
from Classes import Neo4jConnection

# create connection object using neo4j
with open("config.json") as json_data_file:
    data = json.load(json_data_file)
conn = Neo4jConnection(uri=data["uri"], 
                       user=data["user"],              
                       pwd=data["pwd"])


# use py4neo client connection to benchmark
try:
    graph = Graph(data["uri"], auth=(data["user"], data["pwd"]))
    print('SUCCESS: Connected to the Neo4j Database.')
except Exception as e:
    print('ERROR: Could not connect to the Neo4j Database. See console for details.')
    raise SystemExit(e)


# assign directory
directory = data["dir"]
# giving file extension
ext = ('.xpt')
# create empty list to dataset names
dataset_list = list()

# iterate over all files with SAS extension
for files in os.listdir(directory):
    if files.endswith(ext):
        m=files.split(sep='.')[0]
        if type(m) != 'NoneType':
            # append dataset name to list
            dataset_list.append(m)
        # assign datasetname to dataframe name (not a good practice)
        globals()[m] = pd.read_sas(directory+files,format='xport', encoding="utf-8") 
    else:
        continue

Driver successfully created
SUCCESS: Connected to the Neo4j Database.


In [15]:
# testing performance of data migration wrapped in a function using neo4j UNWIND clause
def create_patients_nodes(df):
    # Adds patient nodes to the Neo4j graph.
    query = '''
    UNWIND $rows as row
    CREATE (p:Patient {USUBJID: row.USUBJID, AGE: row.AGE, ARM:row.ARM, SEX: row.SEX, BMI: row.BMIBL})
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})
%timeit create_patients_nodes(adsl)

342 ms ± 73.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
for row in adsl.itertuples():
        graph.run('''
        CREATE (:Patient {USUBJID: $USUBJID, AGE: $AGE, ARM:$ARM, SEX: $SEX, BMI: $BMI})
        ''', parameters = {'USUBJID': row.USUBJID, 'AGE': row.AGE, 'ARM': row.ARM, 'SEX': row.SEX, 'BMI': row.BMIBL})

5.82 s ± 427 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
adsl.to_csv('C:/Temp/dropzone/adsl.csv', encoding='utf-8', index=False)
query = '''
    LOAD CSV WITH HEADERS FROM "file:///C:/Temp/dropzone/adsl.csv" AS row
    CREATE (p:Patient {USUBJID: row.USUBJID, AGE: row.AGE, ARM:row.ARM, SEX: row.SEX, BMI: row.BMIBL})
    '''
conn.query(query, parameters = {})

In [4]:
# The highest performance was obtained with he function wrapping UNWIND clause
# The model will be created using this approach

def create_patients_nodes(df):
    # Adds patient nodes to the Neo4j graph.
    query = '''
    UNWIND $rows as row
    CREATE (p:Patient {USUBJID: row.USUBJID, AGE: row.AGE, ARM:row.ARM, SEX: row.SEX, BMI: row.BMIBL})
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

# unique treatment nodes
treatment_set = set(adsl['ARM'])

# unique adverse event nodes
adversevents = set(adae['AETERM'])
    
def create_visit_nodes(visits_set):
# Adds visit nodes to the Neo4j graph form a python set of unique values
    for i in visits_set:
        query = '''
        CREATE (v:Visit {Name: $Name})
        '''
        return conn.query(query, parameters = {'Name': i})
    
def create_chemlab_nodes(df):
    # Adds chemical laboratory measurements nodes to the Neo4j graph.
    query = '''
    UNWIND $rows as row
    CREATE (p:Parameter {USUBJID: row.USUBJID, VISIT: row.VISIT, Laboratory: row.PARCAT1, Parameter: row.PARAM, Value: row.AVAL, Reference: row.LBNRIND, Dataset: 'adlbc'})
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_hemolab_nodes(df):
    # Adds hematology laboratory measurements nodes to the Neo4j graph.
    query = '''
    UNWIND $rows as row
    CREATE (p:Parameter {USUBJID: row.USUBJID, VISIT: row.VISIT, Laboratory: row.PARCAT1, Parameter: row.PARAM, Value: row.AVAL, Reference: row.LBNRIND, Dataset: 'adlbh'})
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_vitalsigns_nodes(df):
    # Adds vital signs measurements nodes to the Neo4j graph.
    query = '''
    UNWIND $rows as row
    CREATE (vs:VitalSign {USUBJID: row.USUBJID, VISIT: row.VISIT, Laboratory: 'VS', Parameter: row.PARAM, Value: row.AVAL, Reference: '', Dataset: 'advs'})
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_adadas_nodes(df):
    # Adds ADADAS endpoint nodes
    query = '''
    UNWIND $rows as row
    CREATE (ep:Endpoint {USUBJID: row.USUBJID, VISIT: row.VISIT, EndpointName: 'ADAS-Cog', Parameter: row.PARAM, Value: row.AVAL, Reference: '', Dataset: 'adadas'})
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_cibc_nodes(df):
    # Adds ADADAS endpoint nodes
    query = '''
    UNWIND $rows as row
    CREATE (ep:Endpoint {USUBJID: row.USUBJID, VISIT: row.VISIT, EndpointName: 'CIBC Score', Parameter: row.PARAM, Value: row.AVAL, Reference: '', Dataset: 'adcibc'})
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

# Create unique visit names from each dataset

visits_chem = set(adlbc['VISIT'])
visits_vs = set(advs['VISIT'])
visits_hem = set(adlbh['VISIT'])
visits_adadas = set(adadas['VISIT'])
visits_cbic = set(adcibc['VISIT'])
visits = visits_chem | visits_vs | visits_hem | visits_adadas | visits_cbic

# Create relationships

def create_patient_treatment_relationship(df):
    query = '''
    UNWIND $rows as row
    MATCH (p:Patient {USUBJID: row.USUBJID}), (t:Treatment {Name:row.ARM}) 
    MERGE (p)-[r:WAS_TREATED]->(t)
    SET r.Dose=row.TRT01PN
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_patient_adverseevent_relationship(df):
    query = '''
    UNWIND $rows as row
    MATCH (p:Patient {USUBJID: row.USUBJID}), (ae:AdverseEvent {Term: row.AETERM}) 
    MERGE (p)-[r:HAD_ADVERSE_EVENT]->(ae)
    SET r={Severity: row.AESEV, Type: row.AEBODSYS}
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_patient_visit_relationship(df):
    query = '''
    UNWIND $rows as row
    MATCH (p:Patient {USUBJID: row.USUBJID}), (v:Visit {Name: row.VISIT}) 
    MERGE (p)-[:ATTENDED_VISIT]->(v)
    '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_patient_chemistrylab_relationship(df):
    query = '''
    UNWIND $rows as row
    MATCH (p:Patient {USUBJID: row.USUBJID}), (param:Parameter {USUBJID: row.USUBJID, Laboratory: row.PARCAT1,Parameter: row.PARAM, Value: row.AVAL, Reference: row.LBNRIND, Dataset: 'adlbc'}), (v:Visit {Name: row.VISIT})
    WHERE p.USUBJID=row.USUBJID AND param.VISIT=row.VISIT
    CREATE (p)-[lb:MEASURED_LABPARAMETER]->(param)<-[:MEASURED_IN_VISIT]-(v)
    SET lb={ChangeFromBaseline: row.CHG}
        '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_patient_hemolab_relationship(df):
    query = '''
    UNWIND $rows as row
    MATCH (p:Patient {USUBJID: row.USUBJID}), (param:Parameter {USUBJID: row.USUBJID, Laboratory: row.PARCAT1,Parameter: row.PARAM, Value: row.AVAL, Reference: row.LBNRIND, Dataset: 'adlbc'}), (v:Visit {Name: row.VISIT})
    WHERE p.USUBJID=row.USUBJID AND param.VISIT=row.VISIT
    CREATE (p)-[lb:MEASURED_LABPARAMETER]->(param)<-[:MEASURED_IN_VISIT]-(v)
    SET lb={ChangeFromBaseline: row.CHG}
        '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_patient_vitalsign_relationship(df):
    query = '''
    UNWIND $rows as row
    MATCH (p:Patient {USUBJID: row.USUBJID}), (param:VitalSign {USUBJID: row.USUBJID, Laboratory: 'VS' ,Parameter: row.PARAM, Value: row.AVAL, Reference: '', Dataset: 'advs'}), (v:Visit {Name: row.VISIT})
    WHERE p.USUBJID=row.USUBJID AND param.VISIT=row.VISIT
    CREATE (p)-[vs:MEASURED_VITALSIGN]->(param)<-[:MEASURED_IN_VISIT]-(v)
    SET vs={ChangeFromBaseline: row.CHG}
        '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_patient_adadas_relationship(df):
    query = '''
    UNWIND $rows as row
    MATCH (p:Patient {USUBJID: row.USUBJID}), (end:Endpoint {USUBJID: row.USUBJID, VISIT: row.VISIT, EndpointName: 'ADAS-Cog', Parameter: row.PARAM, Value: row.AVAL, Reference: '', Dataset: 'adadas'}), (v:Visit {Name: row.VISIT})
    WHERE p.USUBJID=row.USUBJID AND end.VISIT=row.VISIT
    CREATE (p)-[endrel:ASSESSED_ENDPOINT]->(end)<-[:MEASURED_IN_VISIT]-(v)
    SET endrel={ChangeFromBaseline: row.CHG}
        '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

def create_patient_cibc_relationship(df):
    query = '''
    UNWIND $rows as row
    MATCH (p:Patient {USUBJID: row.USUBJID}), (end:Endpoint {USUBJID: row.USUBJID, VISIT: row.VISIT, EndpointName: 'CIBC Score', Parameter: row.PARAM, Value: row.AVAL, Reference: '', Dataset: 'adcibc'}), (v:Visit {Name: row.VISIT})
    WHERE p.USUBJID=row.USUBJID AND end.VISIT=row.VISIT
    CREATE (p)-[endrel:ASSESSED_ENDPOINT]->(end)<-[:MEASURED_IN_VISIT]-(v)
    SET endrel={ChangeFromBaseline: row.CHG}
        '''
    return conn.query(query, parameters = {'rows': df.to_dict('records')})

In [ ]:
# create nodes from each dataset
create_patients_nodes(adsl)
create_chemlab_nodes(adlbc)
create_hemolab_nodes(adlbh)
create_vitalsigns_nodes(advs)
create_adadas_nodes(adadas)
create_cibc_nodes(adcibc)

# create adverse event nodes
for i in adversevents:
    query = '''
    CREATE (ae:AdverseEvent {Term: $Term})
    '''
    conn.query(query, parameters = {'Term': i})

# create treatment nodes
for i in treatment_set:
    query = '''
    CREATE (t:Treatment {Name: $ARM})
    '''
    conn.query(query, parameters = {'ARM': i})

# create visit nodes
for i in visits:
    query = '''
    CREATE (v:Visit {Name: $Name})
    '''
    conn.query(query, parameters = {'Name': i})

create_patient_treatment_relationship(adsl)
create_patient_adverseevent_relationship(adae)
create_patient_chemistrylab_relationship(adlbc)
create_patient_hemolab_relationship(adlbh)
create_patient_vitalsign_relationship(advs)
create_patient_adadas_relationship(adadas)
create_patient_cibc_relationship(adcibc)